In [1]:
from google.cloud import bigquery

In [2]:
client = bigquery.Client()
project_id = client.project

In [3]:
dataset_id = f"{project_id}.data_to_ai_workshop"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"

dataset = client.create_dataset(dataset, exists_ok=True)

In [4]:
uri = "gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv"
table_id = f"{project_id}.data_to_ai_workshop.emergency_calls_response_times"

In [5]:
job_config = bigquery.LoadJobConfig(
    source_format = bigquery.SourceFormat.CSV,
    skip_leading_rows = 1,
    autodetect = True
)

In [6]:
load_job = client.load_table_from_uri(uri, table_id, job_config = job_config)
load_job.result()

LoadJob<project=qwiklabs-gcp-01-a82571ac193f, location=US, id=abffa92c-9185-46ed-a2ce-e7687df9a349>

In [7]:
%%bigquery
SELECT count(*)
FROM `data_to_ai_workshop.emergency_calls_response_times`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,50000


In [8]:
%%bigquery
SELECT *
FROM `data_to_ai_workshop.emergency_calls_response_times`
LIMIT 5


Query is running:   0%|          |

Downloading:   0%|          |

,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37


In [9]:
%%bigquery
SELECT
    AVG(response_time) as avg_response,
    MIN(response_time) as min_response,
    MAX(response_time) as max_response,
    AVG(distance_to_station) as avg_distance
FROM
    `data_to_ai_workshop.emergency_calls_response_times`;

Query is running:   0%|          |

Downloading:   0%|          |

,avg_response,min_response,max_response,avg_distance
0,17.446134,2.01,36.55,15.079754


In [10]:
%%bigquery
SELECT
    traffic_level,
    AVG(response_time) as avg_response_time,
    COUNT(*) as count
FROM
    `data_to_ai_workshop.emergency_calls_response_times`
GROUP BY
    traffic_level
ORDER BY
    avg_response_time DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,traffic_level,avg_response_time,count
0,High,20.094627,16744
1,Medium,17.138830,16655
2,Low,15.083129,16601


In [11]:
%%bigquery
CREATE OR REPLACE MODEL `data_to_ai_workshop.model_linear`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['response_time']) AS
SELECT
    call_type,
    location,
    weather_condition,
    day_of_week,
    time_of_day,
    traffic_level,
    distance_to_station,
    units_available,
    response_time
FROM
    `data_to_ai_workshop.emergency_calls_response_times`;

Query is running:   0%|          |

""


In [12]:
%%bigquery
CREATE OR REPLACE MODEL `data_to_ai_workshop.model_boosted_tree`
OPTIONS(model_type='BOOSTED_TREE_REGRESSOR',
        input_label_cols=['response_time'],
        max_iterations=10,
        early_stop=true) AS
SELECT
    call_type,
    location,
    weather_condition,
    day_of_week,
    time_of_day,
    traffic_level,
    distance_to_station,
    units_available,
    response_time
FROM
    `data_to_ai_workshop.emergency_calls_response_times`;

Query is running:   0%|          |

""


In [13]:
%%bigquery
CREATE OR REPLACE MODEL `data_to_ai_workshop.model_dnn`
OPTIONS(
    model_type='DNN_REGRESSOR',
    input_label_cols=['response_time'],
    hidden_units=[16],
    max_iterations=10
) AS
SELECT
    call_type,
    location,
    weather_condition,
    day_of_week,
    time_of_day,
    traffic_level,
    distance_to_station,
    units_available,
    response_time
FROM
    `data_to_ai_workshop.emergency_calls_response_times`;

Query is running:   0%|          |

""


In [14]:
%%bigquery
SELECT 'Linear' as model_type, * FROM ML.EVALUATE(MODEL `data_to_ai_workshop.model_linear`);

Query is running:   0%|          |

Downloading:   0%|          |

,model_type,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,Linear,1.761934,4.827846,0.015117,1.501836,0.831417,0.83146


In [15]:
%%bigquery
SELECT 'Boosted Tree' as model_type, * FROM ML.EVALUATE(MODEL `data_to_ai_workshop.model_boosted_tree`);

Query is running:   0%|          |

Downloading:   0%|          |

,model_type,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,Boosted Tree,1.8718,5.522262,0.016942,1.583377,0.807169,0.816221


In [16]:
%%bigquery
SELECT 'DNN' as model_type, * FROM ML.EVALUATE(MODEL `data_to_ai_workshop.model_dnn`);

Query is running:   0%|          |

Downloading:   0%|          |

,model_type,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,DNN,1.7739,4.89387,0.015478,1.508177,0.829111,0.829294


In [17]:
%%bigquery
SELECT
    *
FROM
    ML.PREDICT(MODEL `data_to_ai_workshop.model_linear`,
    (
        SELECT
            'Fire' AS call_type,
            'Downtown' AS location,
            'Rainy' AS weather_condition,
            'Monday' AS day_of_week,
            18 AS time_of_day,
            'High' AS traffic_level,
            15.5 AS distance_to_station,
            2 AS units_available
    ));

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,22.349557,Fire,Downtown,Rainy,Monday,18,High,15.5,2


In [18]:
%%bigquery
SELECT
    *
FROM
    ML.PREDICT(MODEL `data_to_ai_workshop.model_boosted_tree`,
    (
        SELECT
            'Fire' AS call_type,
            'Downtown' AS location,
            'Rainy' AS weather_condition,
            'Monday' AS day_of_week,
            18 AS time_of_day,
            'High' AS traffic_level,
            15.5 AS distance_to_station,
            2 AS units_available
    ));

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,21.776264,Fire,Downtown,Rainy,Monday,18,High,15.5,2


In [19]:
%%bigquery
SELECT
    *
FROM
    ML.PREDICT(MODEL `data_to_ai_workshop.model_dnn`,
    (
        SELECT
            'Fire' AS call_type,
            'Downtown' AS location,
            'Rainy' AS weather_condition,
            'Monday' AS day_of_week,
            18 AS time_of_day,
            'High' AS traffic_level,
            15.5 AS distance_to_station,
            2 AS units_available
    ));

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,22.186567,Fire,Downtown,Rainy,Monday,18,High,15.5,2
